In [2]:
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings(action='ignore')

In [3]:
t_data=pd.read_excel('C:\\Users\\a3011\\OneDrive\\Desktop\\사회조사분석(특성항목)\\2020년 경상남도 사회조사 시군 특성항목 조사자료(수정)\\7. 2020년 경상남도 사회조사 밀양시 특성항목 조사자료(수정).xlsx')

In [4]:
sakin=pd.read_excel('C:\\Users\\a3011\\OneDrive\\Desktop\\사회조사분석(특성항목)\\2020년 경상남도 사회조사 시군 특성항목 조사자료(수정)\\밀양시색인.xlsx')

In [5]:
# 나이 분류
age=t_data['가구원_만나이']
age[age<=19]=1
age[(20<=age) &(age<=29)]=2
age[(30<=age) &(age<=39)]=3
age[(40<=age) &(age<=49)]=4
age[(50<=age) &(age<=59)]=5
age[(60<=age) &(age<=64)]=6
age[65<=age]=7
age
t_data['나이']=age

In [6]:
#학력별
school=t_data[['일반사항_6.응답자_교육정도','일반사항_6-1.응답자_교육정도_수학여부']]

no_graduate=school[school['일반사항_6-1.응답자_교육정도_수학여부']!=1]
graduate=school[school['일반사항_6-1.응답자_교육정도_수학여부']==1]
no_graduate2=no_graduate['일반사항_6.응답자_교육정도']-1

school_2=pd.concat([graduate['일반사항_6.응답자_교육정도'],no_graduate2]).sort_index()
school_2=pd.DataFrame(school_2)
school_2.columns=['교육정도수준']
school_2[school_2['교육정도수준']<=0]=1
school_2[school_2['교육정도수준']>4]=4
t_data['교육정도수준']=school_2

In [7]:
#가구 가중치
gagu_weight=t_data.shape[0]*[79.3719]
t_data['가구 가중치']=gagu_weight

In [8]:
#가구원 가중치
gaguwon_index=t_data[['나이','가구원_성별']]
weight=[(1,1,209.5833),(1,2,209.5833),(1,3,132.8611),(1,4,141.4286),(1,5,105.4396),(1,6,81.0457),(1,7,76.5443),
       (2,1,209.2143),(2,2,209.2143),(2,3,136.9032),(2,4,114.2222),(2,5,100.8495),(2,6,69.5552),(2,7,66.5267)]
for i,j,k in weight:
    gaguwon_index[(gaguwon_index['나이']==j) & (gaguwon_index['가구원_성별']==i)]=k
t_data['가구원 가중치']=gaguwon_index['나이']

In [9]:
t_data[['나이','교육정도수준','가구 가중치','가구원 가중치']]

,나이,교육정도수준,가구 가중치,가구원 가중치
0,3,4,79.3719,132.8611
1,7,3,79.3719,66.5267
2,7,3,79.3719,66.5267
3,5,4,79.3719,105.4396
4,5,4,79.3719,105.4396
...,...,...,...,...
967,6,3,79.3719,69.5552
968,3,4,79.3719,136.9032
969,2,4,79.3719,209.5833
970,7,2,79.3719,76.5443


In [9]:
# 가구원 가중치 데이터
df_index = ['성별','남자','여자','나이','15~19','20~29','30~39','40~49','50~59','60~64','65~','학력','초졸이하','중졸','고졸','대졸이상',
           '거처의종류','단독주택','아파트','연립주택','다세대주택','기타','월평균가구소득','~50','50~','100~','200~','300~','400~','500~','600~','700~',
          '800~','주거점유형태','자기집','전세','보증금있는월세','보증금없는월세','무상']

a = pd.DataFrame(index=range(1,25),columns=df_index)

row_names = ['남자','여자','15~19','20~29','30~39','40~49','50~59','60~64','65~','초졸이하','중졸','고졸','대졸이상',
           '단독주택','아파트','연립주택','다세대주택','기타','~50','50~','100~','200~','300~','400~','500~','600~','700~',
          '800~','자기집','전세','보증금있는월세','보증금없는월세','무상']

identity_category = ['가구원_성별','나이','교육정도수준','일반사항_1.응답자_거처의종류','17-1.가구_월평균가구소득','일반사항_2.응답자_주거점유형태']

sakin2 = sakin

gaguwon_sakin = sakin2[sakin['가중치']=='가구원 가중치']

spl_data = t_data[identity_category]

for i in gaguwon_sakin['변수명']:
    
    spl_df = a.copy()
    if (i in spl_data) == True:
        using_data = spl_data
        using_data = pd.concat([using_data,t_data['가구원 가중치']],axis=1)
    
    else:
        using_data = pd.concat([spl_data,t_data[i]],axis=1)
        using_data = pd.concat([using_data,t_data['가구원 가중치']],axis=1)
    for j in identity_category:
        
        if j=='가구원_성별':
            for idx,k in [('남자',1),('여자',2)]:
                using_data2=using_data[using_data[j] == k]
                data_series=using_data2['가구원 가중치'].groupby(using_data2[i]).sum() / using_data2['가구원 가중치'].groupby(using_data2[i]).sum().sum()*100
                spl_df[idx] = data_series
        elif j=='나이':
            for idx,k in [('15~19',1),('20~29',2),('30~39',3),('40~49',4),('50~59',5),('60~64',6),('65~',7)]:
                using_data2=using_data[using_data[j] == k]
                data_series=using_data2['가구원 가중치'].groupby(using_data2[i]).sum() / using_data2['가구원 가중치'].groupby(using_data2[i]).sum().sum()*100
                spl_df[idx] = data_series
        elif j=='교육정도수준':
            for idx,k in [('초졸이하',1),('중졸',2),('고졸',3),('대졸이상',4)]:
                using_data2=using_data[using_data[j] == k]
                data_series=using_data2['가구원 가중치'].groupby(using_data2[i]).sum() / using_data2['가구원 가중치'].groupby(using_data2[i]).sum().sum()*100
                spl_df[idx] = data_series
        elif j=='일반사항_1.응답자_거처의종류':
            for idx,k in [('단독주택',1),('아파트',2),('연립주택',3),('다세대주택',4),('기타',5)]:
                using_data2=using_data[using_data[j] == k]
                data_series=using_data2['가구원 가중치'].groupby(using_data2[i]).sum() / using_data2['가구원 가중치'].groupby(using_data2[i]).sum().sum()*100
                spl_df[idx] = data_series
        elif j=='17-1.가구_월평균가구소득':
            for idx,k in [('~50',1),('50~',2),('100~',3),('200~',4),('300~',5),('400~',6),('500~',7),('600~',8),('700~',9),('800~',10)]:
                using_data2=using_data[using_data[j] == k]
                data_series=using_data2['가구원 가중치'].groupby(using_data2[i]).sum() / using_data2['가구원 가중치'].groupby(using_data2[i]).sum().sum()*100
                spl_df[idx] = data_series
        elif j=='일반사항_2.응답자_주거점유형태':
            for idx,k in [('자기집',1),('전세',2),('보증금있는월세',3),('보증금없는월세',4),('무상',5)]:
                using_data2=using_data[using_data[j] == k]
                data_series=using_data2['가구원 가중치'].groupby(using_data2[i]).sum() / using_data2['가구원 가중치'].groupby(using_data2[i]).sum().sum()*100
                spl_df[idx] = data_series
        


    spl_df=spl_df.transpose()
    spl_df.to_excel('C:\\Users\\a3011\\OneDrive\\Desktop\\사회조사분석(특성항목)\\특성항목엑셀\\'+'밀양시'+str(i)+'.xlsx')

In [10]:
# 통합데이터 사용
for m in ['통합데이터1']:
    row_names= ['남자','여자','15~19','20~29','30~39','40~49','50~59','60~64','65~','초졸이하','중졸','고졸','대졸이상',
           '단독주택','아파트','연립주택','다세대주택','기타','~50','50~','100~','200~','300~','400~','500~','600~','700~',
          '800~','자기집','전세','보증금있는월세','보증금없는월세','무상']
    df_index=['성별','남자','여자','나이','15~19','20~29','30~39','40~49','50~59','60~64','65~','학력','초졸이하','중졸','고졸','대졸이상',
           '거처의종류','단독주택','아파트','연립주택','다세대주택','기타','월평균가구소득','~50','50~','100~','200~','300~','400~','500~','600~','700~',
          '800~','주거점유형태','자기집','전세','보증금있는월세','보증금없는월세','무상']
    a=pd.DataFrame(index=range(1,25),columns=df_index)
    identity_category=['가구원_성별','나이','교육정도수준','일반사항_1.응답자_거처의종류','17-1.가구_월평균가구소득','일반사항_2.응답자_주거점유형태']
    sakin3=sakin[sakin['가중치']=='가구원 가중치']
    sakin2=sakin3[sakin3['데이터여부']==m]
    save_df=pd.DataFrame()
    for i in sakin2['변수명']:
        abcd=t_data[[i,'가구원 가중치','가구원_성별','나이','교육정도수준','일반사항_1.응답자_거처의종류','17-1.가구_월평균가구소득','일반사항_2.응답자_주거점유형태']]
        
        abcd.columns=['통합정리','가구원 가중치','가구원_성별','나이','교육정도수준','일반사항_1.응답자_거처의종류','17-1.가구_월평균가구소득','일반사항_2.응답자_주거점유형태']
        
        save_df=pd.concat([save_df,abcd])
        
    save_df=save_df[save_df['통합정리'].isnull()==False]
    i='통합정리'
    spl_df=a.copy()
    
    for j in identity_category:
        
        if j=='가구원_성별':
            for idx,k in [('남자',1),('여자',2)]:
                using_data2=save_df[save_df[j] == k]
                data_series=using_data2['가구원 가중치'].groupby(using_data2[i]).sum() / using_data2['가구원 가중치'].groupby(using_data2[i]).sum().sum()*100
                spl_df[idx] = data_series
        elif j=='나이':
            for idx,k in [('15~19',1),('20~29',2),('30~39',3),('40~49',4),('50~59',5),('60~64',6),('65~',7)]:
                using_data2=save_df[save_df[j] == k]
                data_series=using_data2['가구원 가중치'].groupby(using_data2[i]).sum() / using_data2['가구원 가중치'].groupby(using_data2[i]).sum().sum()*100
                spl_df[idx] = data_series
        elif j=='교육정도수준':
            for idx,k in [('초졸이하',1),('중졸',2),('고졸',3),('대졸이상',4)]:
                using_data2=save_df[save_df[j] == k]
                data_series=using_data2['가구원 가중치'].groupby(using_data2[i]).sum() / using_data2['가구원 가중치'].groupby(using_data2[i]).sum().sum()*100
                spl_df[idx] = data_series
        elif j=='일반사항_1.응답자_거처의종류':
            for idx,k in [('단독주택',1),('아파트',2),('연립주택',3),('다세대주택',4),('기타',5)]:
                using_data2=save_df[save_df[j] == k]
                data_series=using_data2['가구원 가중치'].groupby(using_data2[i]).sum() / using_data2['가구원 가중치'].groupby(using_data2[i]).sum().sum()*100
                spl_df[idx] = data_series
        elif j=='17-1.가구_월평균가구소득':
            for idx,k in [('~50',1),('50~',2),('100~',3),('200~',4),('300~',5),('400~',6),('500~',7),('600~',8),('700~',9),('800~',10)]:
                using_data2=save_df[save_df[j] == k]
                data_series=using_data2['가구원 가중치'].groupby(using_data2[i]).sum() / using_data2['가구원 가중치'].groupby(using_data2[i]).sum().sum()*100
                spl_df[idx] = data_series
        elif j=='일반사항_2.응답자_주거점유형태':
            for idx,k in [('자기집',1),('전세',2),('보증금있는월세',3),('보증금없는월세',4),('무상',5)]:
                using_data2=save_df[save_df[j] == k]
                data_series=using_data2['가구원 가중치'].groupby(using_data2[i]).sum() / using_data2['가구원 가중치'].groupby(using_data2[i]).sum().sum()*100
                spl_df[idx] = data_series
       
        
            

    spl_df=spl_df.transpose()
    spl_df.to_excel('C:\\Users\\a3011\\OneDrive\\Desktop\\사회조사분석(특성항목)\\특성항목엑셀\\'+'밀양시(통합데이터)'+str(m)+'.xlsx') 

KeyError: '통합정리'

In [ ]:
/aa['가구원 가중치'].groupby(aa[i]).sum().sum()*100)

In [12]:
for i in sakin['변수명']:
    aa=t_data[[i,'가구원 가중치']]
    print(aa['가구원 가중치'].groupby(aa[i]).sum())

52.밀양시_바람직한도시형태
1    33794.3615
2    19791.7216
3    13446.7616
4    16409.3867
5    10504.2365
6      100.8495
Name: 가구원 가중치, dtype: float64
53.밀양시_관광객유치견해
1    33990.8928
2    25811.9201
3    11397.2304
4    10882.4854
5    11568.9867
6      395.8020
Name: 가구원 가중치, dtype: float64
54-1.밀양시_희망문화공연_0
1    28870.0100
2     5811.0229
3    15292.5336
4     8741.2361
5     2013.4041
6     3534.4949
7    29784.6158
Name: 가구원 가중치, dtype: float64
54-1.밀양시_희망문화공연_1
1    19861.9923
2     7830.3796
3    17785.7494
4    10207.1166
5     7079.9318
6     4392.3031
7    26813.3003
8       76.5443
Name: 가구원 가중치, dtype: float64
54-2.밀양시_희망문화예술교육프로그램_0
1    41170.3504
2    16883.3409
3     3186.2588
4    13945.1382
5    18781.1834
6       81.0457
Name: 가구원 가중치, dtype: float64
54-2.밀양시_희망문화예술교육프로그램_1
1    18743.9735
2    18556.4411
3    11342.7087
4    14954.2894
5    30449.9047
Name: 가구원 가중치, dtype: float64
54-3.밀양시_희망전시프로그램_0
1    33018.7748
2    17783.6097
3    10238.8989
4    19793.8045
5     6583.79